## FaceRecognizer
#### The Following Code was trained on Google Colab because of GPU restrictions
#### Input: 
- Training data from drive (or local disk if training from local disk in the future)
- Testing data from drive (or local disk if training from local disk in the future)

#### Output:
- Predicted Label corresonding to image

In [0]:
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import os
from os.path import isdir
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier


In [0]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    #if image is not having face
    if len(results)==0:
        return []
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [0]:
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
    # if image is not having any face
    if len(face) == 0:
        continue
        # store
        faces.append(face)
    return faces

In [0]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

#### Loading Dataset from FaceDetectorTrain and FaceDetectorTest from directory

In [11]:
trainX, trainy = load_dataset("../data/FaceDetectorTrain/")
testX, testy = load_dataset("../data/FaceDetectorTest/")
print(trainX.shape)
print(testX.shape)
print(trainy.shape)
print(testy.shape)


(561, 160, 160, 3)
(118, 160, 160, 3)
(561,)
(118,)


#### keraspath variable is used to load the model, as we are using facenet model (pretrained model ) 
- facenet_keras.h5 pretrained model that was used was downloaded from manually 

In [0]:
keraspath="../model/facenet_keras.h5"
from keras.models import load_model
model = load_model(keraspath,compile=False)

In [0]:
# used to convert iamge of shape(160*160*3) into vectors(1 * 128)
def get_embedding(model, face_pixels):
  # converting to float data type
    face_pixels = face_pixels.astype('float32')
  # finding mean,std of face
    (mean, std) = (face_pixels.mean(), face_pixels.std())
  # Normalizing the vector
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [18]:
#convert each image present in trainX to embedded vector 
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

(561, 128)


In [19]:
#convert each image present in testX to embedded vector
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

(118, 128)


#### We used Random forest classifier to train our data

In [22]:
trainX=newTrainX
trainy=trainy
testX=newTestX
testy=testy
concat = np.concatenate((trainX,testX),axis=0)

lent = len(trainX)

in_encoder = Normalizer(norm='l2')
concat = in_encoder.transform(concat)

trainX = concat[:lent]
testX = concat[lent:]

clf = RandomForestClassifier(n_estimators = 500,n_jobs=-1)
clf.fit(trainX,trainy)
predicted = clf.predict(testX)
print(accuracy_score(testy,predicted))

0.9915254237288136


#### We saved pickle files corresponds to the following on our local machines, to save time during further computations:
- training data
- training label
- testing data
- testing label
- classifier (it is an object of type random forest)
- model ( which is pretrained )

#### The directory structures in the above code have been changed to use relative paths with respect to the final codes directory structure for future scenarios of training